In [1]:
%matplotlib inline
from pylab import *
import pandas as pd
from sklearn import manifold
from sklearn import *

import numpy as np 
from numpy import *

C:\Users\User\Anaconda3\lib\site-packages\sklearn\lda.py:4: DeprecationWarning: lda.LDA has been moved to discriminant_analysis.LinearDiscriminantAnalysis in 0.17 and will be removed in 0.19
  "in 0.17 and will be removed in 0.19", DeprecationWarning)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\qda.py:4: DeprecationWarning: qda.QDA has been moved to discriminant_analysis.QuadraticDiscriminantAnalysis in 0.17 and will be removed in 0.19.
  "in 0.17 and will be removed in 0.19.", DeprecationWarning)


 <b>Avec le prétraitement en R  des variables qualitatives et des valeurs manquantes  : <b>

In [2]:

x_train= pd.read_csv("traincomplet.csv",sep=",")
y_train=pd.read_csv("sortie_train.csv",sep=",")
x_test=pd.read_csv("test.csv",sep=",")



 <b>affichage des premieres lignes de input_train, input_test et output_train<b> :

In [3]:

x_train.head()

,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,...,Q63,Q64,Q65,Q66,Q67,Q68,Q69,Q70,Q71,Q72
0,2,3,2,0,0,0,1,1,8,2286,...,0,0,0,1,0,1,0,1.0,0,0
1,2,3,1,0,0,0,2,1,166,2220,...,0,0,0,0,0,1,0,1.0,0,0
2,4,2,1,0,0,0,2,1,17,2265,...,0,0,0,0,0,1,0,1.0,0,0
3,2,3,2,0,0,0,2,1,16,2250,...,0,0,1,0,0,1,0,1.0,0,0
4,2,2,2,0,0,0,1,1,23,2305,...,0,0,0,1,0,0,1,1.0,0,0


In [4]:
y_train.head()

,ID,TARGET
0,0,0
1,1,0
2,2,0
3,3,1
4,4,0


In [5]:
x_test.head()

,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,...,Q63,Q64,Q65,Q66,Q67,Q68,Q69,Q70,Q71,Q72
0,2,2,1,0,0,0,1,1,128,2295,...,0,0,0,0,1,1,0,1,0,0
1,2,2,2,0,1,0,1,1,20,2275,...,0,0,0,1,0,1,0,0,1,0
2,2,2,2,0,0,0,1,1,4,2320,...,0,0,1,0,0,0,1,0,0,0
3,2,2,2,1,0,0,1,1,6,2277,...,0,0,0,0,0,1,1,1,0,0
4,2,4,2,0,0,0,2,1,9,2245,...,0,0,0,0,1,1,0,1,0,0


<b>sélection des variables et transformation de l'échantillon d'entrainement  <b>

In [6]:

#sélection des meilleures variables la classifiaction :
from sklearn.feature_selection import SelectKBest, f_classif
# objet sklearn pour faire cette reduction de dim
Selec = SelectKBest(f_classif, k=50)
#x_train=x_train.values


#y_train
#selection = Selec.fit_transform(x_train,y_train)






In [7]:
y_train=y_train.values[:,1]
shape(y_train)

(85529,)

In [8]:
x_train=x_train.values


In [9]:
selection = Selec.fit_transform(x_train,y_train)
selection

array([[ 3.,  2.,  0., ...,  1.,  0.,  0.],
       [ 3.,  1.,  0., ...,  1.,  0.,  0.],
       [ 2.,  1.,  0., ...,  1.,  0.,  0.],
       ..., 
       [ 2.,  1.,  1., ...,  1.,  0.,  0.],
       [ 3.,  2.,  0., ...,  1.,  0.,  0.],
       [ 2.,  2.,  1., ...,  1.,  0.,  0.]])

<b>stratification de l'echantillon: 
En effet un test nous a montré qu'on a vait autant de 1 que 0 à predire alors que l'échantillon d'apprentissage n'a pas du tout cette Stratification d'ou par une méthode de tirage aléatoire nous allons constituer une échantillon stratifié<b>

In [10]:

#identification des lignes avecdes 0 et des 1
ind0=[0]*85530
for i in range(85529):
    ind0[i]=i*(y_train[i]==0)

ind0
v1=[0]
v2=[3]
for i in range(85529):
    if ind0[i]>0:
        v1.append(ind0[i])
    elif (ind0[i]==0)&(i>3):
        v2.append(i)
        
ind0=v1
ind1=v2
len(ind1)

from random import uniform
v=[0]*len(ind1)
for i in range(len(ind1)):
    u=uniform(0,1)
    v[i]=(u<0.5)*ind0[i]+(u>=0.5)*ind1[i]

tir=v
len(tir)


16558

<b>prediction par plus proche voisins sur l'echantillon stratifié avec les variables sélectionnées :<b>

In [11]:


from sklearn import neighbors
mod = neighbors.KNeighborsClassifier(n_neighbors = 10, weights="uniform")
mod.fit(selection[tir[1:10000]], y_train[tir[1:10000]])
#cross validation  avec un bloc plus petit  :
#print("Taux de bien classés sur le 2ème bloc :") 
print(mod.score(selection[tir[10001:12000]], y_train[tir[10001:12000]]))



0.555277638819


In [12]:


x_test=x_test.values
selection_pred = Selec.transform(x_test)
#selection_pred
shape(x_test)
shape(x_train)

(85529, 85)

In [13]:
pred = mod.predict(selection_pred)
pred
sum(pred)#pour vérifier que la proportion de 1 prédit est bien en accord 

8136

<b>même avec une stratification les k plus proches voisins ont des resultats relativements mauvais <b>

<b>Prédiction par Support vecteur machine :<b>

In [14]:
#test du svm posons le modèle:


#from sklearn import svm
#mod_svm = svm.SVC(kernel='rbf')

#mod_svm.fit(selection[tir[1:12000],:], y_train[tir[1:12000]])
#pred_svm = mod_svm.predict(x_train[tir[1:12000],:])
#print("Taux de bien classés sur les données d'apprentissage :") 
#print(mod_svm.score(selection[tir[12001:14000],:], y_train[tir[12001:14000]]))

In [15]:
#pred_test1=mod_svm.predict(x_test)
#sum(pred_test1)

 <b>création du modèle de Random forest  avec les variables sélectionnées et l'echantillon stratifié   <b>

In [16]:

#création du modèle avec les variables sélectionnées et l'echantillon stratifié  :

from sklearn.ensemble import RandomForestClassifier
mod3= RandomForestClassifier(n_estimators=200,max_features=50,max_depth =101,criterion="entropy")
mod3.fit(selection[tir[0:9000],:], y_train[tir[0:9000]])

#mini-cross validation avec deux blocs plus petits :
print("Taux de bien classés sur les données de validation :") 

print(mod3.score(selection[tir[9001:11622],:], y_train[tir[9001:11622]]))

#print(mod3.score(selection[tir[12001:13622],:], y_train[tir[12001:13622]]))

Taux de bien classés sur les données de validation :
0.632201449828


<b>creation de fichiers de soumission <b>

In [ ]:

#pred_test2=mod3.predict(x_test)
#tab=pd.DataFrame(pred_test2,columns=["TARGETS"])
#tab.to_csv('pyt.csv', header=True)
#sum(pred_test2)


 <b>sélection de variables sur la random forest (les variables les plus importantes ) <b>:

In [17]:
imp=mod3.feature_importances_
imp
p=[]
type(p)
for i in range(50):
    if imp[i]>0.01:
         p.append(i)
p
len(p)

28

In [18]:
imp

array([ 0.02189237,  0.02972597,  0.00743659,  0.00546916,  0.00339793,
        0.03824114,  0.00742909,  0.01321412,  0.00446775,  0.01664508,
        0.03804677,  0.06726047,  0.03953223,  0.01918829,  0.04914848,
        0.07103345,  0.04194409,  0.02361171,  0.0369302 ,  0.00432895,
        0.00471556,  0.03784901,  0.02308155,  0.00863815,  0.02034182,
        0.05537269,  0.02096606,  0.02024129,  0.01721728,  0.02184637,
        0.0360041 ,  0.02448374,  0.00543101,  0.02632949,  0.01375632,
        0.00582265,  0.01329363,  0.00801115,  0.00992273,  0.02167457,
        0.00680131,  0.00459916,  0.0085813 ,  0.00773257,  0.00860612,
        0.00190454,  0.00696299,  0.00850578,  0.00512067,  0.00724254])

 <b>   entrainnement sur l'échantillon constitué de variables les plus importantes dans la random_forest:  <b>

In [19]:

from sklearn.ensemble import RandomForestClassifier
x_train_new=selection[:,p]
mod4= RandomForestClassifier(n_estimators=200,max_features=len(p),max_depth =101,criterion="entropy")
mod4.fit(x_train_new[tir[0:9000],:], y_train[tir[0:9000]])

#cross-validation sur un bloc plus petit :
print("Taux de bien classés sur les données de validation :") 

print(mod4.score(x_train_new[tir[9001:11622],], y_train[tir[9001:11622]]))

Taux de bien classés sur les données de validation :
0.619610835559


 <b>Création du fichier de soumission  <b>:

In [20]:
pred_test3=mod4.predict(x_test[:,p])
sum(pred_test3)
tab=pd.DataFrame(pred_test3,columns=["TARGETS"])
tab.to_csv('pyt.csv', header=True)
sum(pred_test3)


21224

<b>  mise en oeuvre du Gradient Boosting pour booster la performance des arbres succéssifs  :<b> 

In [21]:

from sklearn.ensemble import GradientBoostingClassifier
mod5= GradientBoostingClassifier(n_estimators=100,max_features=50,max_depth =61,verbose=True)
mod5.fit(selection[tir[0:9000],:], y_train[tir[0:9000]])
#mod5_appr = mod5.predict(x_train[tir[0:8000],:])
print("Taux de bien classés sur les données d'apprentissage :") 
#calcul du score sur deux echantillons de test (cross validation de la methode) avec 
#la même strat

#Cross-validation des paramètres (non automatisée car de multiple test de paramètres non fortuits ont étés éffectués )
print(mod5.score(selection[tir[9001:12000],:], y_train[tir[9001:12000]]))



      Iter       Train Loss   Remaining Time 
         1           1.1963            1.76m
         2           1.0406            1.72m
         3           0.9109            1.70m
         4           0.8012            1.68m
         5           0.7075            1.66m
         6           0.6268            1.64m
         7           0.5568            1.62m
         8           0.4957            1.60m
         9           0.4422            1.58m
        10           0.3951            1.56m
        20           0.1355            1.38m
        30           0.0487            1.23m
        40           0.0178            1.06m
        50           0.0065           52.51s
        60           0.0024           41.75s
        70           0.0009           30.89s
        80           0.0006           19.02s
        90           0.0006            8.49s
       100           0.0006            0.00s
Taux de bien classés sur les données d'apprentissage :
0.568522840947


In [22]:
#validation sur un deuxième echantillon de même taille que le premier:


print(mod5.score(selection[tir[12000:14000],:], y_train[tir[12000:14000]]))

0.579


 <b>création d'un fichier de soumission <b>

In [23]:

#pred_test2=mod5.predict(x_test)
#sum(pred_test2)
#tab=pd.DataFrame(pred_test2,columns=["TARGETS"])
#tab.to_csv('pyt.csv', header=True)
#sum(pred_test2)



